In [2]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ───────────────────────────────────────────────────────────────
# 1) 파일 경로 설정 (본인 환경에 맞게 확인)
REG_DATA_FP   = '/Users/gun/Desktop/미래에셋 AI 공모전/data/module_datasets/regression_enriched.csv'
CLF_DATA_FP   = '/Users/gun/Desktop/미래에셋 AI 공모전/data/module_datasets/classification_with_text.csv'
PRED_FP       = '/Users/gun/Desktop/미래에셋 AI 공모전/data/results/regression/regression_predictions_for_ensemble.csv'
CLF_MODEL_FP  = '/Users/gun/Desktop/미래에셋 AI 공모전/data/models/lgbm_classifier.pkl'
MASTER_CSV    = '/Users/gun/Desktop/미래에셋 AI 공모전/all_stocks_master.csv'

# ───────────────────────────────────────────────────────────────
# 2) 데이터 로드
df_reg  = pd.read_csv(REG_DATA_FP,   parse_dates=["rcept_dt"], dtype={"stock_code":str})
df_clf  = pd.read_csv(CLF_DATA_FP,   parse_dates=["rcept_dt"], dtype={"stock_code":str})
df_pred = pd.read_csv(PRED_FP,       parse_dates=["rcept_dt"], dtype={"stock_code":str})

# ───────────────────────────────────────────────────────────────
# 3) 분류 확률(p_up) 계산
clf = joblib.load(CLF_MODEL_FP)
X_clf = df_clf.drop(columns=["up_1d","corp_name","stock_code","rcept_dt"], errors="ignore")
df_clf["p_up"] = clf.predict_proba(X_clf)[:, 1]

# ───────────────────────────────────────────────────────────────
# 4) 클러스터 라벨 생성
scaler = StandardScaler()
scaled_feats = scaler.fit_transform(df_pred[["y_pred", "residual"]])
df_pred["cluster"] = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(scaled_feats)

# ───────────────────────────────────────────────────────────────
# 5) 마스터 데이터 병합
df_master = (
    df_pred
    .merge(df_clf[["stock_code", "rcept_dt", "p_up"]],
           on=["stock_code", "rcept_dt"], how="left")
    .merge(df_reg, on=["stock_code", "rcept_dt"], how="left", suffixes=("", "_orig"))
)

# ───────────────────────────────────────────────────────────────
# 6) CSV 저장
df_master.to_csv(MASTER_CSV, index=False, encoding="utf-8-sig")
print(f"✅ Master CSV saved → {MASTER_CSV}")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmu

✅ Master CSV saved → /Users/gun/Desktop/미래에셋 AI 공모전/all_stocks_master.csv
